<a href="https://colab.research.google.com/github/diamondleng/Inaccessible-pore-volume/blob/main/IAPV_Supervised_Learning_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IAPV Prediction


In this project, we use supervised learning models to build a regression model for IAPV. Furthermore, we will analyze top factors that influence the treatment result. [Dataset information](https://drive.google.com/file/d/13XfQiUvm_nylzBhK23MKRVsYm2ZZlCwq/view?usp=sharing).

## Contents


* [Part 1: Data Exploration](#Part-1:-Data-Exploration)
* [Part 2: Feature Preprocessing](#Part-2:-Feature-Preprocessing)
* [Part 3: Model Training and Results Evaluation](#Part-3:-Model-Training-and-Result-Evaluation)

# Part 0: Setup Google Drive Environment / Data Collection
check this [link](https://colab.research.google.com/notebooks/io.ipynb) for more info

In [ ]:
import pandas as pd
df = pd.read_csv('Dataforlearning_IAPV.csv')
df.head()

,"IPV, %","Ka, md",Porosity,"Mole Weight, MMdalton","Concentration, ppm",Resistance Factor,Frr,"Retention, ug/g","Flow Velocity, ft/D"
0,1.8,300,0.18,2.0,943,10.05,1.0,42.83,0.1
1,4.0,90,0.16,7.0,1000,10.41,1.0,43.82,10.0
2,6.0,30,0.18,2.0,943,10.05,1.0,92.00,0.1
3,9.0,1600,0.25,2.0,1560,14.05,1.0,54.82,5.0
4,10.0,2000,0.25,20.0,1000,10.41,1.0,43.82,5.0


In [ ]:
df.describe()

,"IPV, %","Ka, md",Porosity,"Mole Weight, MMdalton","Concentration, ppm",Resistance Factor,Frr,"Retention, ug/g","Flow Velocity, ft/D"
count,49.000000,49.000000,49.000000,49.000000,49.00000,49.000000,49.000000,49.000000,49.000000
mean,22.609796,819.755102,0.226122,10.704082,875.77551,10.376531,3.642449,43.722245,2.434694
std,9.980429,1315.336594,0.048939,7.787489,880.57127,7.319852,2.432736,29.555393,2.564465
min,1.800000,30.000000,0.120000,2.000000,49.00000,1.000000,1.000000,9.000000,0.100000
25%,17.000000,277.000000,0.190000,5.500000,400.00000,5.870000,1.500000,27.500000,0.100000
50%,21.380000,408.000000,0.210000,5.500000,600.00000,8.830000,3.970000,37.340000,1.000000
75%,29.880000,660.000000,0.250000,20.000000,1000.00000,10.850000,4.520000,43.820000,5.000000
max,49.000000,7683.000000,0.410000,30.000000,5500.00000,39.690000,16.000000,215.000000,10.000000


# Part 1: Data Exploration

### Part 1.1: Understand the Raw Dataset

In [ ]:
import pandas as pd
import numpy as np

ISG_df = df

In [ ]:
ISG_df.head()

,"IPV, %","Ka, md",Porosity,"Mole Weight, dalton","Concentration, ppm",Resistance Factor,Frr,"Retention, ug/g","Flow Velocity, ft/D"
0,4.0,90.0,0.34275,5500000.0,250,9.0,4.42,32.46,0.1
1,0.0,120.0,0.34275,5500000.0,250,9.0,4.42,32.46,0.1
2,15.0,4500.0,0.34800,5500000.0,500,25.0,4.28,35.87,0.1
3,23.0,5650.0,0.34300,5500000.0,250,9.0,4.42,32.46,0.1
4,26.0,4900.0,0.33900,5500000.0,125,4.0,4.49,30.88,0.1


In [ ]:
# check data info
ISG_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   IPV, %               127 non-null    float64
 1   Ka, md               127 non-null    float64
 2   Porosity             127 non-null    float64
 3   Mole Weight, dalton  127 non-null    float64
 4   Concentration, ppm   127 non-null    int64  
 5   Resistance Factor    127 non-null    float64
 6   Frr                  127 non-null    float64
 7   Retention, ug/g      127 non-null    float64
 8   Flow Velocity, ft/D  127 non-null    float64
dtypes: float64(8), int64(1)
memory usage: 9.1 KB


import math

In [ ]:
# check data info
ISG_df.describe()

,"IPV, %","Ka, md",Porosity,"Mole Weight, dalton","Concentration, ppm",Resistance Factor,Frr,"Retention, ug/g","Flow Velocity, ft/D"
count,127.000000,127.000000,127.000000,1.270000e+02,127.000000,127.000000,127.000000,127.000000,127.000000
mean,22.498189,2689.119685,0.250065,1.125197e+07,943.314961,10.033937,3.326535,52.307638,4.783465
std,8.950739,5264.563370,0.053610,7.572107e+06,968.946120,11.402782,3.115937,55.756177,5.708804
min,0.000000,30.000000,0.116327,2.000000e+06,49.000000,1.000000,1.000000,9.000000,0.100000
25%,17.300000,277.000000,0.205800,5.500000e+06,400.000000,4.300000,1.000000,32.460000,0.100000
50%,22.000000,660.000000,0.250065,5.500000e+06,757.000000,7.160000,3.570000,43.820000,2.000000
75%,27.030000,4875.000000,0.266500,2.000000e+07,1250.000000,10.630000,4.420000,48.420000,5.000000
max,49.000000,50000.000000,0.418000,3.000000e+07,5500.000000,99.100000,23.290000,530.000000,24.000000


In [ ]:
ISG_df['K2']=ISG_df['K']*ISG_df['K']
ISG_df['Porosity2']=ISG_df['Porosity']*ISG_df['Porosity']
ISG_df['Concentration2']=ISG_df['Concentration']*ISG_df['Concentration']
ISG_df['Resistance_Factor2']=ISG_df['Resistance_Factor']*ISG_df['Resistance_Factor']
ISG_df['Oil_Viscosity2']=ISG_df['Oil_Viscosity']*ISG_df['Oil_Viscosity']
ISG_df['Retention2']=ISG_df['Retention']*ISG_df['Retention']
ISG_df['Flow_Velocity2']=ISG_df['Flow_Velocity']*ISG_df['Flow_Velocity']

ISG_df['Klog']=np.log2(ISG_df['K'])
ISG_df['Porositylog']=np.log2(ISG_df['Porosity'])
ISG_df['Concentrationlog']=np.log2(ISG_df['Concentration'])
ISG_df['Resistance_Factorlog']=np.log2(ISG_df['Resistance_Factor'])
ISG_df['Oil_Viscositylog']=np.log2(ISG_df['Oil_Viscosity'])
ISG_df['Retentionlog']=np.log2(ISG_df['Retention'])
ISG_df['Flow_Velocitylog']=np.log2(ISG_df['Flow_Velocity'])




In [ ]:
# understand Numerical feature
ISG_df.describe()

In [ ]:
# check the feature distribution
# pandas.DataFrame.describe()
# boxplot, distplot, countplot
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.boxplot(x='Flow_Velocity',data=ISG_df,orient='horizontal',width=0.8)

In [ ]:
Bulk_Viscosity=[25,	9,	4,	7,	4.1	,3.7	,2.45	,1.65	,2.95,	6,	6,	8,	4.43,	4.08,	4.78,	3.89,	3.98,	3.5,	3.88,	3.88,	6.68,	6.68,	6.47,	6.47,	5.91,	5.91,	5.8,	5.8,	7.2,	7.2,	7.24,	7.24	,7.4,	7.4,	7.67,	7.67]
sns.boxplot(x=Bulk_Viscosity,orient='horizontal',width=0.8)

In [ ]:
ISG_df_H = ISG_df.loc[ISG_df['Polymer'] == 'HPAM']
ISG_df_X = ISG_df.loc[ISG_df['Polymer'] == 'Biopolymer']
ISG_df_O = ISG_df.loc[ISG_df['Sor'] == 'Yes']
ISG_df_W = ISG_df.loc[ISG_df['Sor'] == 'No']
ISG_df=ISG_df_H

In [ ]:
_,axss = plt.subplots(2,2, figsize=[20,10])
#00 HPAM, 01 Biopolymer, 10 Oil, 11 No Oil
sns.boxplot(y='IAPV',data=ISG_df_H,ax=axss[0][0])
sns.boxplot(y='IAPV',data=ISG_df_X,ax=axss[0][1])
sns.boxplot(y='IAPV', data=ISG_df_O,ax=axss[1][0])
sns.boxplot(y='IAPV', data=ISG_df_W,ax=axss[1][1])

In [ ]:
# scatterplot for numerical feature
_,axss = plt.subplots(4,3, figsize=[20,10])
p1=sns.scatterplot(x='IAPV', y ='K', data=ISG_df_H,  ax=axss[0][0])
p2=sns.scatterplot(x='IAPV', y ='Porosity', data=ISG_df_H,  ax=axss[0][1])
p3=sns.scatterplot(x='IAPV', y ='Salinity', data=ISG_df_H,  ax=axss[0][2])
p4=sns.scatterplot(x='IAPV', y ='Sor', data=ISG_df_H,  ax=axss[1][0])
p5=sns.scatterplot(x='IAPV', y ='Mole_Weight', data=ISG_df_H, ax=axss[1][1])
p6=sns.scatterplot(x='IAPV', y ='Concentration', data=ISG_df_H,  ax=axss[1][2])
p7=sns.scatterplot(x='IAPV', y ='Polymer', data=ISG_df_H,  ax=axss[2][0])
p8=sns.scatterplot(x='IAPV', y ='Resistance_Factor', data=ISG_df_H,  ax=axss[2][1])
p9=sns.scatterplot(x='IAPV', y ='Oil_Viscosity', data=ISG_df_H,  ax=axss[2][2])
p10=sns.scatterplot(x='IAPV', y ='Frr', data=ISG_df_H,  ax=axss[3][0])
p11=sns.scatterplot(x='IAPV', y ='Retention', data=ISG_df_H,  ax=axss[3][1])
p12=sns.scatterplot(x='IAPV', y ='Flow_Velocity', data=ISG_df_H,  ax=axss[3][2])


#可以根据每个图对比提出hyposises，后续可以进行验证

In [ ]:
# scatterplot for numerical feature
_,axss = plt.subplots(4,3, figsize=[20,10])
p1=sns.scatterplot(x='IAPV', y ='K', data=ISG_df_X,  ax=axss[0][0])
p2=sns.scatterplot(x='IAPV', y ='Porosity', data=ISG_df_X,  ax=axss[0][1])
p3=sns.scatterplot(x='IAPV', y ='Salinity', data=ISG_df_X,  ax=axss[0][2])
p4=sns.scatterplot(x='IAPV', y ='Sor', data=ISG_df_X,  ax=axss[1][0])
p5=sns.scatterplot(x='IAPV', y ='Mole_Weight', data=ISG_df_X, ax=axss[1][1])
p6=sns.scatterplot(x='IAPV', y ='Concentration', data=ISG_df_X,  ax=axss[1][2])
p7=sns.scatterplot(x='IAPV', y ='Polymer', data=ISG_df_X,  ax=axss[2][0])
p8=sns.scatterplot(x='IAPV', y ='Resistance_Factor', data=ISG_df_X,  ax=axss[2][1])
p9=sns.scatterplot(x='IAPV', y ='Oil_Viscosity', data=ISG_df_X,  ax=axss[2][2])
p10=sns.scatterplot(x='IAPV', y ='Frr', data=ISG_df_X,  ax=axss[3][0])
p11=sns.scatterplot(x='IAPV', y ='Retention', data=ISG_df_X,  ax=axss[3][1])
p12=sns.scatterplot(x='IAPV', y ='Flow_Velocity', data=ISG_df_X,  ax=axss[3][2])

# Part 2: Feature Preprocessing

In [ ]:
ISG_df=ISG_df.dropna(axis=0)
ISG_df


In [ ]:
# Get feature space by dropping useless feature
to_drop = ['IAPV','Polymer','Salinity','Mole_Weight','Frr','Sor']
X = ISG_df.drop(to_drop, axis=1)


In [ ]:
to_drop2 = ['IAPV','Polymer','Salinity','Mole_Weight','Frr','Flow_Velocity','Sor']
X_tilt = ISG_df.drop(to_drop2, axis=1)

In [ ]:
X.head()

In [ ]:
X_tilt.head()

In [ ]:
X.dtypes

In [ ]:
X_tilt.dtypes

In [ ]:
cat_cols = X.columns[X.dtypes == 'O']#categorical type is 0
num_cols = X.columns[(X.dtypes == 'float64') | (X.dtypes == 'int64')]
cat_cols_noq = X.columns[X.dtypes == 'O']#categorical type is 0
num_cols_noq = X.columns[(X.dtypes == 'float64') | (X.dtypes == 'int64')]

In [ ]:
num_cols

In [ ]:
num_cols_noq

In [ ]:
cat_cols

#Split dataset
#应该先分数据，再进行preprocessing，因为testing set数据尽量不要动


In [ ]:
# Objective: y is Treatment Result
y=ISG_df['IAPV']

In [ ]:
# Splite data into training and testing
# 100 -> 75:y=1, 25:y=0
# training(80): 60 y=1; 20 y=0 
# testing(20):  15 y=1; 5 y=0

from sklearn import model_selection

# Reserve 25% for testing
# stratify example:
# 100 -> y: 80 '0', 20 '1' -> 4:1
# 80% training 64: '0', 16:'1' -> 4:1
# 20% testing  16:'0', 4: '1' -> 4:1
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=1) 

print('training data has ' + str(X_train.shape[0]) + ' observation with ' + str(X_train.shape[1]) + ' features')
print('test data has ' + str(X_test.shape[0]) + ' observation with ' + str(X_test.shape[1]) + ' features')

In [ ]:
# No flow rate
X_train_noq, X_test_noq, y_train_noq, y_test_noq = model_selection.train_test_split(X_tilt, y, test_size=0.2, random_state=1) #stratify = y :=> stratified sampling 分层抽样 根据y的分类进行分层然后随机抽样

print('training data has ' + str(X_train_noq.shape[0]) + ' observation with ' + str(X_train_noq.shape[1]) + ' features')
print('test data has ' + str(X_test_noq.shape[0]) + ' observation with ' + str(X_test_noq.shape[1]) + ' features')

Read more for handling [categorical feature](https://github.com/scikit-learn-contrib/categorical-encoding), and there is an awesome package for [encoding](http://contrib.scikit-learn.org/category_encoders/).

In [ ]:
# One hot encoding(convert string to numbers)
# another way: get_dummies
from sklearn.preprocessing import OneHotEncoder

def OneHotEncoding(df, enc, categories):  
  transformed = pd.DataFrame(enc.transform(df[categories]).toarray(), columns=enc.get_feature_names(categories))
  return pd.concat([df.reset_index(drop=True), transformed], axis=1).drop(categories, axis=1)

categories = ['Sor']
enc_ohe = OneHotEncoder()
enc_ohe.fit(X_train[categories])
#有时候test和train内的categories不同，如果先treat后分就会涉及到了test set所以要先分
X_train = OneHotEncoding(X_train, enc_ohe, categories)
X_test = OneHotEncoding(X_test, enc_ohe, categories)
######################################################################################################


In [ ]:
X_train.head()

In [ ]:
categories = ['Sor']
enc_ohe = OneHotEncoder()
enc_ohe.fit(X_train_noq[categories])
#有时候test和train内的categories不同，如果先treat后分就会涉及到了test set所以要先分
X_train_noq = OneHotEncoding(X_train_noq, enc_ohe, categories)
X_test_noq = OneHotEncoding(X_test_noq, enc_ohe, categories)
######################################################################################################


In [ ]:
X_train_noq.head()

Standardize/Normalize Data

In [ ]:
# Save a copy of original dataset
X_train_s=X_train
X_train_noq_s=X_train_noq

In [ ]:
# Scale the data, using standardization
# standardization (x-mean)/std
# normalization (x-x_min)/(x_max-x_min) ->[0,1]

# 1. speed up gradient descent
# 2. same scale
# 3. algorithm requirments

# for example, use training data to train the standardscaler to get mean and std 
# apply mean and std to both training and testing data.
# fit_transform does the training and applying, transform only does applying.
# Because we can't use any info from test, and we need to do the same modification
# to testing data as well as training data

# https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py
# https://scikit-learn.org/stable/modules/preprocessing.html


# min-max example: (x-x_min)/(x_max-x_min)
# [1,2,3,4,5,6,100] -> fit(min:1, max:6) (scalar.min = 1, scalar.max = 6) -> transform [(1-1)/(6-1),(2-1)/(6-1)..]
# scalar.fit(train) -> min:1, max:100
# scalar.transform(apply to x) -> apply min:1, max:100 to X_train
# scalar.transform -> apply min:1, max:100 to X_test

# scalar.fit -> mean:1, std:100
# scalar.transform -> apply mean:1, std:100 to X_train
# scalar.transform -> apply mean:1, std:100 to X_test

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

In [ ]:
scaler.fit(X_train_noq)
X_train_noq = scaler.transform(X_train_noq)
X_test_noq = scaler.transform(X_test_noq)

# Part 3: Model Training and Result Evaluation

### Part 3.1: Model Training

In [ ]:
#@title build models
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

# Logistic Regression
regression_linear = LinearRegression()

# K Nearest Neighbors
regression_KNN = KNeighborsRegressor()

# Random Forest
regression_RF = RandomForestRegressor()

In [ ]:
# Train the model
r1=regression_linear.fit(X_train, y_train)

coeff=r1.coef_
varname=X_train_s.columns
matrce=[coeff,varname]
sns.barplot(x=coeff,y=varname,orient='horizontal')

# Prediction of test data
y1=r1.predict(X_test)
# Accuracy of test data
r1.score(X_test, y_test)

In [ ]:
importances = regression_linear.feature_importances_

indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature importance ranking by Linear Model:")
for ind in range(X.shape[1]):
  print ("{0} : {1}".format(X_train_s.columns[indices[ind]],round(importances[indices[ind]], 4)))

In [ ]:
# Train the model no flow rate
r2=regression_linear.fit(X_train_noq, y_train_noq)
coeff=r2.coef_
varname=X_train_noq_s.columns
matrce=[coeff,varname]
sns.barplot(x=coeff,y=varname,orient='horizontal')
# Prediction of test data
y2=r2.predict(X_test_noq)

r2.score(X_test_noq, y_test_noq)

In [ ]:
# Use 5-fold Cross Validation to get the accuracy for different models
model_names = ['Regression_linear','KNN','Random Forest']
model_list = [regression_linear, regression_KNN, regression_RF]
count = 0

for model in model_list:
    cv_score = model_selection.cross_val_score(model, X_train, y_train, cv=5)
    print(cv_score)
    print('Model accuracy of ' + model_names[count] + ' is ' + str(cv_score.mean()))
    count += 1

In [ ]:
df.loc[df['column_name'] == some_value]